In [14]:
import sys
import cv2 as cv
import ipywidgets as widgets
import matplotlib.pyplot as plt
sys.path.append('../src/interactive_kit')

car = cv.imread("../images/car_pad.tif", cv.IMREAD_UNCHANGED).astype('float64')

In [15]:
import numpy as np
from imageviewer import ImageViewer as viewer
%matplotlib widget

In [16]:
img = np.random.normal(size = (100, 100))

In [17]:
viewer(img, hist = True)

In [18]:
T_slider = widgets.IntSlider(value = 200, min = 0, max = 255, step = 1, description = 'T' )
button = widgets.Button(description = 'Solarize')

# Define solarize
def solarize(img, T = None):
    if T == None:
        T = T_slider.value
    output = np.copy(img)
    output[output < T] = 255-(255-T)*output[output<T]/T
    return output


# Make JS version a numpy array
solarized_pier = solarize(car, 200)

view = viewer([car, solarized_pier], title=['Original', 'Solarized'], new_widgets=[T_slider, button], callbacks=[solarize, np.array], subplots=[1, 2])
button.click()
view.compare_callback()

Button(description='Show Widgets', style=ButtonStyle())

In [19]:
# Function that computes the 2D sinusoidal plane wave at location (x, y) given the parameters A, T and alpha
def cos2D(x, y, A, T, alpha):
    s = 0
    
    ### BEGIN SOLUTIONS
    s = A * np.cos(2 * np.pi / T * (x * np.cos(alpha) + y * np.sin(alpha)))
    ### END SOLUTIONS
    
    return s

# Creates a 2D sinusoidal plane wave of a given size and with given parameters
def create_wave(A, T, alpha, shape=(256, 256), deg=False):
    # Convert degrees to radians
    alpha = alpha/180 * np.pi if deg else alpha
    # Apply sin2D to the whole image
    return np.fromfunction(lambda y, x: cos2D(x, y, A=A, T=T, alpha=alpha), shape=shape)

# Define sliders and button
A_slider = widgets.FloatSlider(value=1, min=0, max=1, step=0.01, description='A')
T_slider = widgets.IntSlider(value=32, min=3, max=256, step=1, description='T')
alpha_slider = widgets.IntSlider(value=0, min=-90, max=90, step=1, description=r'$\alpha [deg]$')
button = widgets.Button(description='Create 2D sin')

# Callback function that applies the vignette effect
def sin2D_callback(img):
    return create_wave(A_slider.value, T_slider.value, alpha_slider.value, deg=True)

plt.close('all')
view = viewer(create_wave(A=1, T=32, alpha=0, deg=True), 
              new_widgets=[A_slider, T_slider, alpha_slider, button], 
              callbacks=[sin2D_callback], widgets=True, clip_range=[-1, 1])